In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
import pickle
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from utils.prompts import  contextualize_q_prompt, qa_prompt
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [3]:
from langchain_community.utilities import SQLDatabase
mysql_uri = 'mysql+mysqlconnector://root:@127.0.0.1:3306/products'
db = SQLDatabase.from_uri(mysql_uri)

In [4]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [5]:
def get_schema(_):
    schema = db.get_table_info()
    return schema
get_schema(None)


'\nCREATE TABLE cb_products (\n\t`ID` BIGINT(20) UNSIGNED NOT NULL, \n\tcode VARCHAR(255) NOT NULL, \n\tname TEXT, \n\tdescription TEXT, \n\turl VARCHAR(255), \n\timage_url TEXT, \n\tcategory_names TEXT, \n\tcategory_label TEXT, \n\ttag_names TEXT, \n\tPRIMARY KEY (`ID`)\n)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci\n\n/*\n3 rows from cb_products table:\nID\tcode\tname\tdescription\turl\timage_url\tcategory_names\tcategory_label\ttag_names\n9237\tB4705N\tSHOWER HEAD TRONICO 250\t• Contemporary organic • Ø 250 mm – L-M 415 • Intelligent water distribution resulting in minimal wa\thttps://www.nikles.com/product/shower-head-tronico-250\thttps://www.nikles.com/wp-content/uploads/2020/05/B4705N1-scaled.jpg\tHead Showers\tTronico\tAirdrop, Easy-to-clean\n9251\tBLS.001.07N\tLIPS CASCADE SHOWER HEAD - MATTE BLACK\t• Outstanding contemporary • Dimensions: 215 x 82.5 • Cascade spray with intelligent water • Materia\thttps://www.nikles.com/product/lips-cascade-shower-head-bla

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


In [7]:
user_question = 'how many shower heads are in the database?'
sql_chain.invoke({"question": user_question})



"SELECT COUNT(*) \nFROM cb_products \nWHERE description LIKE '%SHOWER HEAD%';"

In [8]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response. DO NOT ever 
mention the table name in the question or response. DO NOT mention the sql query. Act like the query is for a shop that produces some products and you are the owner.
Be consciese and accurate in your response.
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [9]:
def run_query(query):
    return db.run(query)


In [10]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
)
def sql_qery(input):
    return full_chain.invoke({"question": input})


In [11]:
user_question = 'tell me about different shower heads available'
full_chain.invoke({"question": user_question}).pretty_print()

# 'There are 347 albums in the database.'


================================== Ai Message ==================================

We have a variety of shower heads available for purchase. Some of them include the 'SHOWER HEAD TECHNO 65 UNO', 'SHOWER HEAD TECHNO 65 3S MASSAGE', 'SHOWER HEAD XL TECHNO 180', 'SHOWER HEAD XL TECHNO 200 BRASS', 'SHOWER HEAD FRESH LED ROUND 300', 'HEAD SHOWER FRESH ROUND 105 UNO', 'HEAD SHOWER FRESH ROUND 105 DUO', 'HEAD SHOWER FRESH ROUND 180', 'SHOWER HEAD FRESH ROUND 200', 'SHOWER HEAD FRESH SQUARE 200', 'SHOWER HEAD AVANTI 115 DUO', 'SHOWER HEAD AVANTI 115 DUO SILK', 'INOX RAINFRAME 300 TECHNO KIT', 'SHOWER HEAD DROP 75 2S SPRAY', 'SHOWER HEAD DROP 75 UNO', 'SHOWER HEAD TECHNO FRESH 95 3S MASSAGE', 'SHOWER HEAD TECHNO FRESH 95 UNO', 'SHOWER HEAD FRESH ROUND 250', 'SHOWER HEAD FRESH SQUARE 250', 'SHOWER HEAD ROUND 250 DROPLESS', 'SHOWER HEAD SQUARE 250 DROPLESS', 'SHOWER HEAD ROUND 300 DROPLESS', 'SHOWER HEAD SQUARE 300 DROPLESS', 'SHOWER HEAD AVANTI ROUND 250 DROPLESS', 'SHOWER HEAD AVANTI QUADRO 250 

In [12]:
def run_sql(input):
    return full_chain.invoke(input)

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
system_prompt = """
You are an assistant for specific question-answering tasks based on given context.

you always follow these guidelines:

- if the answer is not in the context, say that you can't help.
- if the question is any history or generel knowledge based question that is not relevant to the context, say that you can't help.
- if the question is not in the context, say that you can't help.
- Check that if the question is relevant to the context. If not, say that you can't help.
- If you don't know the answer, say that you don't know.
- The question and answer that you generated must be in the context, otherwise saythat you can't help.
- Use three sentences maximum and keep the answer concise.


{context}
"""
# system_prompt = (
#     "You are an assistant for specific question-answering tasks based on given context."
#     "Use the following pieces of retrieved context to answer "
#     "DO NOT say anything thats not related to the context. If the question and answer is not in the context, say that you can't help."
#     "the question. If the question and answer is not in the context, say that you can't help."
#     "Check that if the question is relevant to the context. If not, say that you can't help."
#     "If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [21]:
# with open('db.pkl','rb') as f: pkl = pickle.load(f)
    
# vectorstore = FAISS.deserialize_from_bytes(serialized=pkl, embeddings=OpenAIEmbeddings())
vectorstore = FAISS.load_local("embedded_knowledge_base",embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
llm=ChatOpenAI(model="gpt-3.5-turbo")

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=history_aware_retriever
)

# conversational_rag_chain = RunnablePassthrough(
#     rag_chain,
#     # get_session_history,
#     input_messages_key="input",
#     history_messages_key="chat_history",
#     output_messages_key="answer",
# )
# conversational_rag_chain = RunnablePassthrough(
#         rag_chain,
#         get_session_history,
#         input_messages_key="input",
#         history_messages_key="chat_history",
#         output_messages_key="answer",
#     )

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract method _get_relevant_documents (type=type_error)

In [40]:
retriever.invoke("when bangladesh independent")

[Document(page_content='1982\n\n1990 - 1995\n\n1996 - 2000\n\n2000\n\n2007\n\n2008\n\n2009\n\n2010\n\n2012\n\n2013\n\n2014\n\n2015\n\n2017\n\n2018\n\n2021\n\n2015 -\n\nWHY CHOOSE US?\n\nSCHEDULE\n\nWe follow a coherent schedule with each client.\n\nHIGH QUALITY PRODUCTS\n\nWe provide products according to international standards.\n\nHIGH QUALITY SERVICES\n\nWe provide varied business services.\n\nINTERNATIONAL COMPANY\n\nOur branches are located in Switzerland, Italy, Germany, Thailand, China and USA.\n\nSCHEDULE\n\nWe follow a coherent schedule with each client.\n\nHIGH QUALITY PRODUCTS\n\nWe provide products according to international standards.\n\nHIGH QUALITY SERVICES\n\nWe provide varied business services.\n\nINTERNATIONAL COMPANY\n\nOur branches are located in Switzerland, Italy, Germany, Thailand, China and USA.\n\nSCHEDULE\n\nWe follow a coherent schedule with each client.\n\nHIGH QUALITY PRODUCTS\n\nWe provide products according to international standards.\n\nHIGH QUALITY SERV

In [41]:
conversational_rag_chain.invoke("capital of france")

{'query': 'capital of france', 'result': 'The capital of France is Paris.'}

In [42]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [43]:
router_template="""
    You are an inteliigent router. Given the input below, classify is as one of the following: product, service 
    if the input is about anything related to products, say products.
    if the input is more generalizd, like history, technologies, news, categories etc, say service.

    Do not generate more than one word.

    <input>
    {input}
    </input>

    Classification:
"""

router_chain = (
    PromptTemplate.from_template(router_template) 
    | llm
    | StrOutputParser()
)

router_chain.invoke("capital of spain")

'service'

In [44]:
# user_question = "What is the capital of India?"
# router_response = router_chain.invoke(user_question)
# # final_answer = router_response
# if( router_response=='service'):
    

In [52]:
from langchain.agents import Tool

product_tool = Tool(
    name = "database",
    func=sql_qery,
    description="Use this tool when ansering about any product or object, anything related to products, product description, whenever you think it is necessary to search a database, or when you are not sure",
)
docs_tool = Tool(
    name= "website",
    func=qa.invoke,
    description="Use this tool when ansering about technologies, history, and warranties and news, DO NOT use it if any kind of product related question is asked",
    
)


In [70]:
agent_promt = "You must only use the provided tools to answer questions. If you cannot use a tool to answer, respond with 'I don't know.' Do not generate answers directly."
agent_promt = ChatPromptTemplate.from_messages(
    [
        ("system", agent_promt),
        ("human", "{input}"),
    ]
)
from langchain.agents import initialize_agent
tools = [product_tool, docs_tool]
conv_agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    agent_kwargs={
        'prefix': "Assistant is an agent that will use two tools. And if the question is not from these tools it will say that it is sorry. It will not generate answers that the tools cannot generate. It does not have a knowledgebase itself. Anything it doesn't know, it will say sorry."
    }
)


In [69]:
# conv_agent.agent.llm_chain.prompt.messages = [
#     SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Assistant is an agent that will use two tools. And if the question is not from these tools it will say that it is sorry. It will not generate answers that the tools cannot generate. It does not have a knowledgebase itself. Anything it doesn't know, it will say sorry.")),
#  MessagesPlaceholder(variable_name='chat_history'),
#  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> database: Use this tool when ansering about any product or object, anything related to products, product description, whenever you think it is necessary to search a database, or when you are not sure\n> website: Use this tool when ansering about technologies, history, and warranties and news, DO NOT use it if any kind of product related question is asked\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    'action': string, \\\\ The action to take. Must be one of database, website\n    'action_input': string \\\\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:\n\n```json\n{{\n    'action': 'Final Answer',\n    'action_input': string \\\\ You should put what you want to return to use here\n}}\n```\n\nUSER\'S INPUT\n--------------------\nHere is the user\'s input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else):\n\n{input}")),
#  MessagesPlaceholder(variable_name='agent_scratchpad')]


NameError: name 'SystemMessagePromptTemplate' is not defined

In [71]:
conv_agent.invoke("what are different shower heads options")



> Entering new AgentExecutor chain...
```json
{
    "action": "database",
    "action_input": "different types of shower heads"
}
```

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 366d9027-5407-4938-805b-4fcb7a180c0f, but expected {'tool'} run.")



Observation: content="We have a variety of shower heads available in the shop categorized under 'Head Showers'. Some of the types include SHOWER HEAD TECHNO 65 UNO, SHOWER HEAD XL TECHNO 180, SHOWER HEAD FRESH LED ROUND 300, and many more." response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 828, 'total_tokens': 882}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-77920d8e-334e-4796-8186-a66137f438a8-0'
Thought:```json
{
    "action": "Final Answer",
    "action_input": "We have a variety of shower heads available in the shop categorized under 'Head Showers'. Some of the types include SHOWER HEAD TECHNO 65 UNO, SHOWER HEAD XL TECHNO 180, SHOWER HEAD FRESH LED ROUND 300, and many more."
}
```

> Finished chain.


{'input': 'what are different shower heads options',
 'chat_history': [HumanMessage(content='what are different shower heads options'),
  AIMessage(content='There are different types of shower heads available, including Nikles cascade showers, multifunction showers with click mechanisms, shower heads with dropless systems, and showers with Nikles booster technology. Additionally, there are features like Nikles Light for illumination, Nikles Sound for playing music, and Nikles Carbon shower heads made from carbon fiber. Each type offers unique functions and benefits for a rejuvenating shower experience.'),
  HumanMessage(content='what are different shower heads options'),
  AIMessage(content='There are two different types of shower heads available: Tronico and Lips.')],
 'output': "We have a variety of shower heads available in the shop categorized under 'Head Showers'. Some of the types include SHOWER HEAD TECHNO 65 UNO, SHOWER HEAD XL TECHNO 180, SHOWER HEAD FRESH LED ROUND 300, and m

: 

In [47]:
from langchain_core.messages import SystemMessage
from langgraph.checkpoint import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

system_message = "You must only use the provided tools to answer questions. If you cannot use a tool to answer, respond with 'I don't know.' Do not generate answers directly."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

memory = MemorySaver()
app = create_react_agent(
    llm, tools, messages_modifier=system_message, checkpointer=memory,
)

In [48]:
config = {"configurable": {"thread_id": "test-thread"}}
print(
    app.invoke(
        {
            "messages": [
                ("user", "capital of spain")
            ]
        },
        config,
    )["messages"][-1].content
)

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 27bc9dc9-a138-4a80-a28d-da02d12a75d4, but expected {'tool'} run.")


The capital of Spain is Madrid.




> Entering new AgentExecutor chain...
```json
{
    "action": "website",
    "action_input": "different types of shower heads"
}
```

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID db1ef947-1ea8-4e39-a29b-4fc6fa1a8ce4, but expected {'tool'} run.")



Observation: {'query': 'different types of shower heads', 'result': 'There are different types of shower heads available, including Nikles cascade showers, multifunction showers with click mechanisms, shower heads with dropless systems, and showers with Nikles booster technology. Additionally, there are features like Nikles Light for illumination, Nikles Sound for playing music, and Nikles Carbon shower heads made from carbon fiber. Each type offers unique functions and benefits for a rejuvenating shower experience.'}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "There are different types of shower heads available, including Nikles cascade showers, multifunction showers with click mechanisms, shower heads with dropless systems, and showers with Nikles booster technology. Additionally, there are features like Nikles Light for illumination, Nikles Sound for playing music, and Nikles Carbon shower heads made from carbon fiber. Each type offers unique functions and 

{'input': 'what are different shower heads options',
 'chat_history': [],
 'output': 'There are different types of shower heads available, including Nikles cascade showers, multifunction showers with click mechanisms, shower heads with dropless systems, and showers with Nikles booster technology. Additionally, there are features like Nikles Light for illumination, Nikles Sound for playing music, and Nikles Carbon shower heads made from carbon fiber. Each type offers unique functions and benefits for a rejuvenating shower experience.'}

In [50]:
from langchain.prompts import ChatPromptTemplate

agent_prompt = "You must only use the provided tools to answer questions. If you cannot use a tool to answer, respond with 'I don't know.' Do not generate answers directly."

agent_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", agent_prompt),
        ("human", "{input}"),
    ]
)
from langchain.agents import create_tool, create_agent
from langchain.chains.conversation.memory import ConversationBufferMemory

# Assuming docs_tool and product_tool are defined
tools = [docs_tool, product_tool]

# Create a custom agent that strictly uses tools
class ToolOnlyAgent:
    def __init__(self, tools, llm, memory, prompt_template, verbose=True):
        self.tools = tools
        self.llm = llm
        self.memory = memory
        self.prompt_template = prompt_template
        self.verbose = verbose

    def run(self, input_text):
        # Prepare the prompt with input
        prompt = self.prompt_template.format(input=input_text)
        
        # Get the agent's response
        response = self.llm(prompt)

        # Check if the response is trying to provide a final answer
        if "Final Answer" in response:
            return "I don't know."
        
        # Otherwise, proceed with using the tools
        for tool in self.tools:
            tool_response = tool(input_text)
            if tool_response:
                return tool_response

        # If no tool could provide an answer
        return "I don't know."

# Initialize the custom agent
agent = ToolOnlyAgent(
    tools=tools,
    llm=llm,
    memory=conversational_memory,
    prompt_template=agent_prompt_template,
    verbose=True
)

# Use the custom agent
response = agent.run("When is Bangladesh Independence Day?")
print(response)


ImportError: cannot import name 'create_tool' from 'langchain.agents' (c:\Users\BS-Test\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\agents\__init__.py)

In [ ]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Who will the Bears draft first in the NFL draft?"}
    )
)
print(question_router.invoke({"question": "What are the types of agent memory?"}))

In [ ]:
from langchain.agents import AgentType
print(AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION.prompt.template) 

AttributeError: 'AgentType' object has no attribute 'prompt'

In [ ]:
tools=[conversational_rag_chain,full_chain]
model_with_tools = llm.bind_tools(tools)

ValueError: Unsupported function

bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=RunnableAssign(mapper={
    context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
             | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000260F7FC6350>, search_kwargs={'k': 6}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, lang

In [ ]:
very_big_chain = conversational_rag_chain | full_chain 

In [ ]:
very_big_chain.invoke({"input":"when nikles created"},
                      config={
                "configurable": {"session_id":f"{1}"}
            })

Parent run 65fc5b75-dcce-461f-a797-af75f47438c7 not found for run a2869a14-933f-44fd-8da6-4f39d1537557. Treating as a root run.


KeyError: "Input to ChatPromptTemplate is missing variables {'question'}.  Expected: ['question', 'schema'] Received: ['input', 'chat_history', 'context', 'answer', 'schema']"

In [ ]:
tool_prompt = """ You are the owner of a company named Nikles. Answer the following question. you have access to the following tools:

conversationa_rag_chain: A chain that retrieves data or warranties of products, and some information about the company.
full_chain : A chain that generates information about different products 

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [conversationa_rag_chain, full_chain]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

"""

In [ ]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model_with_tools, tools, tool_prompt)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
agent = initialize_agent(tools=[conversational_rag_chain,full_chain],
                        llm=llm,
                        agent="zero-shot-react-description",
                        verbose =True
                        )

c:\Users\BS-Test\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


AttributeError: 'RunnableWithMessageHistory' object has no attribute 'is_single_input'